## CONF test

In [22]:
%run -v0
[20]
text1 = text0 + '''
simulate: R()
    $x: x
    @CONF: work_dir = /tmp, exec_path = /tmp
'''
res = test_format(ds, text1)
assert res.modules['simulate'].path == ['/tmp']

In [23]:
%run -v0
[25]
text1 = text0 + '''
simulate: R()
    $x: x
    @CONF: work_dir = /tmp, exec_path = (/tmp, ~/tmp), lib_path = ./, R_libs = (ashr@stephenslab/ashr (2.2.7+), psych)
'''
res = test_format(ds, text1)
assert res.modules['simulate'].path == ['/tmp', '~/tmp']
assert res.modules['simulate'].exe['header'] == 'library(ashr)\nlibrary(psych)'

## Input / output variable test

In [24]:
%run -v0
[21]
# F: multiple input/output
text1 = text0 + '''
simulate: R()
    $x: x, 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: R()
    x: $x, y
    $x: x
'''
res = test_format(ds, text1)

* Module output ``x`` cannot contain multiple elements ``['x', 7]``
* Module input ``x`` cannot contain multiple elements ``['$x', 'y']``


## Duplicates

In [25]:
%run -v1
[22]
# F: various duplicates
text1 = text0 + '''
simulate: R()
    $x: x

simulate, simulate: R()
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: R()
    $x: x

simulate: R()
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: R()
    $x: x

t,t: R()
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: R()
    $x: x

t,simulate: R()
    $x: 7
'''
res = test_format(ds, text1)
text1 = text0 + '''
simulate: R()
    $x: x
    $x: y

t: R()
    $x: 7
'''
res = test_format(ds, text1)

* Duplicate module in block ``simulate,simulate``.


WARNING: Overwriting existing module definition simulate ...

* Duplicate module in block ``t,t``.
* Duplicate module definition ``simulate``
* Duplicate key is not allowed: ``$x``, in DSC configuration:
``simulate:
    $x: x
    $x: y``


## FILTER test

In [26]:
%run -v0
[23]
# F: bad condition
text = text0 + '''
simulate: R()
    n: $y
    $x: x
    @FILTER: n < 3
'''
res = test_format(ds, text)
text = text0 + '''
simulate: R()
    n: 2
    p: 5
    $x: x
    @FILTER: n < 3
    @ALIAS: m = n
'''
res = test_format(ds, text)

* Invalid @FILTER: ``['n < 3']``!


In [27]:
%run -v0
[24]
# P: filter tests
text = text0 + '''
simulate, t: R(), R()
    n: 100, 200, 300, 400, 500
    k: 0, 1
    @FILTER:
        simulate: (n <= 300 and k = 0) or (n > 300 and k = 1)
        t: n = 500
    $x: x
'''
res = test_format(ds, text)
assert (res.modules['simulate'].dump()['input_filter']) == '(_n <= 300 and _k == 0) or (_n > 300 and _k == 1)'
text = text0 + '''
simulate, t: R(), R()
    n: 100, 200, 300, 400, 500
    k: 0, 1
    @FILTER: (n in [100,200,300] and k = 0)
    $x: x
'''
res = test_format(ds, text)
assert (res.modules['simulate'].dump()['input_filter']) == '(_n in (100,200,300) and _k == 0)'
text = text0 + '''
simulate, t: R(), R()
    n: 100, 200, 300, 400, 500
    k: 0, 1
    @FILTER: 
        *: n in [100,200,300]
        t: n = 300
    $x: x
'''
res = test_format(ds, text)
assert (res.modules['simulate'].dump()['input_filter']) == '(_n in (100,200,300))'

In [28]:
%run -v0
[26]
# F: parameter name conflict with output
text = text0 + '''
simulate: R()
    n: 100
    $n: n
'''
res = test_format(ds, text)
# F: parameter name conflict with output
text = text0 + '''
simulate: R()
    n: $n
    $n: n
'''
res = test_format(ds, text)

* Name ``n`` cannot be used for both parameter and output for module ``simulate``


In [29]:
%run -v0
[27]
# F: parameter name conflict with output
text = text0 + '''
simulate: R()
    n: 100
    a: 5
    @ALIAS: m = n, m = a
    $out: n
'''
res = test_format(ds, text)

* Duplicated @ALIAS ``['m']`` in module ``simulate``


## File()

In [30]:
%run -v0
[28]
# P: file() operator
text = text0 + '''
simulate: R()
    data: file(.txt)
    $out: n
'''
res = test_format(ds, text)

[29]
# F: file() operator misuage
text = text0 + '''
simulate: R()
    data: file(.txt), 1
    $out: n
'''
res = test_format(ds, text)

[30]
# F: file() operator misuage
text = text0 + '''
simulate: R()
    data: (file(.txt), 1)
    $out: n
'''
res = test_format(ds, text)

[31]
# F: bad output
text = text0 + '''
simulate: R()
    data: file(.txt)
    $out: data, 1
'''
res = test_format(ds, text)

[32]
# F: bad output
text = text0 + '''
simulate: R()
    data: file(.txt)
    $out: (data, 1)
'''
res = test_format(ds, text)

* Cannot mix file operator ``file(.txt)`` with other values ``[1]``!
* File operator inside tuple ``('file(.txt)', 1)`` is not allowed!
* Module output ``out`` cannot contain multiple elements ``['data', 1]``
* Module output ``out`` cannot contain multiple elements ``[('data', 1)]``


## quotes keywords

In [31]:
%run -v0
[33]
text = text0 + '''
simulate: R()
    K: TRUE, FALSE, NULL
    $out: K
'''
res = test_format(ds, text)
assert res.modules['simulate'].dump()['input']['K'] == ['TRUE', 'FALSE', 'NULL']

[34]
text = text0 + '''
simulate: Python()
    K: 'TRUE', 'FALSE', 'NULL'
    $out: K
'''
res = test_format(ds, text)
assert res.modules['simulate'].dump()['input']['K'] == ["'TRUE'", "'FALSE'", "'NULL'"]